In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import time
from yahooquery import Ticker

In [2]:

# ==========================================
# 1. STEP 1: FETCH UNIVERSE
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")
    return list(set(tickers))

In [3]:
# ==========================================
# 2. STEP 2: FINANCIALS ONLY FILTER
# ==========================================
def filter_financials_universe(ticker_list):
    print(f"\n--- STEP 2: Filtering for Financial Services (YahooQuery) ---")
    
    MIN_PRICE = 5.0
    MIN_CAP_US = 300_000_000
    MIN_VOL_US = 2_000_000 # Kept as per your provided code
    
    valid_candidates = []
    chunk_size = 500
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=False)
            data = yq.get_modules("summaryProfile summaryDetail price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # Sector Check
                profile = data[symbol].get('summaryProfile', {})
                sector = profile.get('sector', 'Unknown')
                if 'Financial' not in sector and 'Real Estate' not in sector: continue

                # Basic Data
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                
                fin_mod = data[symbol].get('financialData', {})
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key and isinstance(rec_key, str): rec_key = rec_key.lower().strip()
                
                # Filters
                if curr_price < MIN_PRICE: continue
                if mkt_cap < MIN_CAP_US: continue
                if avg_vol < MIN_VOL_US: continue
                if rec_key not in ['buy', 'strong_buy', 'strong buy']: continue

                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Sector': sector,
                    'Rating': rec_key
                })
        except:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

In [4]:
# ==========================================
# 3. DATA ENRICHMENT
# ==========================================
def fetch_financial_data(df):
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return None

    print(f"\n--- STEP 3: FETCHING METRICS FOR {len(df)} STOCKS ---")
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        if index % 5 == 0: print(f"   Fetching {index}/{len(df)}...", end='\r')
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            recom = info.get('recommendationMean', None)
            
            bank_data.append({
                'Ticker': ticker,
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': float(recom) if recom is not None else 3.0,
                'Sector': row.get('Sector', 'Financial')
            })
        except: continue
            
    master_df = pd.DataFrame(bank_data)
    
    # Force numeric types
    cols = ['P/E', 'P/B', 'ROE', 'Yield%', 'Recom']
    for col in cols: master_df[col] = pd.to_numeric(master_df[col], errors='coerce')
    
    return master_df

In [5]:
# ==========================================
# 4. VIEW: VALUE PICKS
# ==========================================
def get_value_picks(df):
    mask = (df['P/E'] < 15) & (df['P/B'] < 1.2) & (df['ROE'] > 0.08)
    value_df = df[mask].copy()
    value_df = value_df.sort_values(by='P/B', ascending=True)
    cols = ['Ticker', 'Price', 'P/B', 'P/E', 'ROE', 'Yield%', 'Recom']
    return value_df[cols]

# ==========================================
# 5. VIEW: INCOME PICKS
# ==========================================
def get_income_picks(df):
    mask = (df['Yield%'] >= 2.5) & (df['P/E'] < 20) & (df['Recom'] <= 2.5)
    income_df = df[mask].copy()
    income_df = income_df.sort_values(by='Yield%', ascending=False)
    cols = ['Ticker', 'Price', 'Yield%', 'P/E', 'Recom', 'Sector']
    return income_df[cols]

# ==========================================
# 6. VIEW: GROWTH PICKS (NEW)
# ==========================================
def get_growth_picks(df):
    """
    Returns stocks with High P/E (>= 20) and Strong Buy rating (<= 1.5).
    """
    # Filter: P/E >= 20, Rating <= 1.5 (Strong Buy)
    mask = (df['P/E'] >= 20) & (df['Recom'] <= 1.5)
    growth_df = df[mask].copy()
    
    # Sort by Rating (Lowest number is best rating)
    growth_df = growth_df.sort_values(by='Recom', ascending=True)
    
    cols = ['Ticker', 'Price', 'P/E', 'Recom', 'Yield%', 'Sector']
    return growth_df[cols]



In [6]:
# ==========================================
# EXECUTION BLOCK
# ==========================================
raw_tickers = get_raw_universe()
financial_df = filter_financials_universe(raw_tickers)

if not financial_df.empty:
    master_df = fetch_financial_data(financial_df)
    
    if master_df is not None and not master_df.empty:
        # Create all 3 DataFrames
        df_value = get_value_picks(master_df)
        df_income = get_income_picks(master_df)
        df_growth = get_growth_picks(master_df) # <--- New DF
        
        print("\n\n✅ Done! Three DataFrames are ready for Data Wrangler:")
        print("   1. df_value  (Undervalued)")
        print("   2. df_income (High Dividend)")
        print("   3. df_growth (High P/E & Strong Buy)")
        
        print("\n--- Growth Preview ---")
        print(df_growth.head())

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.

--- STEP 2: Filtering for Financial Services (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 100

--- STEP 3: FETCHING METRICS FOR 100 STOCKS ---
   Fetching 95/100...

✅ Done! Three DataFrames are ready for Data Wrangler:
   1. df_value  (Undervalued)
   2. df_income (High Dividend)
   3. df_growth (High P/E & Strong Buy)

--- Growth Preview ---
   Ticker   Price        P/E    Recom  Yield%              Sector
34    BUR    9.09  22.725000  1.00000   138.0  Financial Services
5    RIOT   13.44  25.846153  1.27778     0.0  Financial Services
74    HUT   49.64  25.587627  1.33333     0.0  Financial Services
17      V  355.00  34.838078  1.50000    75.0  Financial Services
23    KKR  130.41  54.564854  1.50000    57.0  Financial Services


In [7]:
import os
import time
import pandas as pd

# ==========================================
# 4. SEPARATE SAVE FUNCTION (Updated)
# ==========================================
def save_financials_to_excel(df_value, df_income, df_growth, master_df):
    """
    Saves the specific 'Picks' dataframes to OneDrive.
    """
    # 1. Setup Filename
    today_date = time.strftime("%Y-%m-%d")
    file_nickname = f"Financial_Stock_Picks_{today_date}.xlsx"
    
    # 2. Setup Folder (Explicit Path)
    onedrive_folder = r"C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks"
    full_path = os.path.join(onedrive_folder, file_nickname)

    print(f"Saving to: {full_path}...")

    try:
        if not os.path.exists(onedrive_folder):
            os.makedirs(onedrive_folder, exist_ok=True)

        with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
            # Sheet 1: Value
            if df_value is not None and not df_value.empty:
                df_value.to_excel(writer, sheet_name='Value Picks', index=False)
            
            # Sheet 2: Income
            if df_income is not None and not df_income.empty:
                df_income.to_excel(writer, sheet_name='Income Picks', index=False)
                
            # Sheet 3: Growth (New)
            if df_growth is not None and not df_growth.empty:
                df_growth.to_excel(writer, sheet_name='Growth Picks', index=False)
            
            # Sheet 4: Master Data
            if master_df is not None and not master_df.empty:
                master_df.to_excel(writer, sheet_name='All Data', index=False)
            
        print(f"✅ Success! Excel file created.")
        print(f"   - Value Picks: {len(df_value) if df_value is not None else 0}")
        print(f"   - Income Picks: {len(df_income) if df_income is not None else 0}")
        print(f"   - Growth Picks: {len(df_growth) if df_growth is not None else 0}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Close the Excel file if it is currently open.")

# --- TRIGGER SAVE ---
# We check if the dataframes exist in your environment before saving
if 'df_value' in locals() and 'df_income' in locals() and 'df_growth' in locals():
    save_financials_to_excel(df_value, df_income, df_growth, master_df)
else:
    print("⚠️ DataFrames are missing. Please run the analysis code block first.")

Saving to: C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks\Financial_Stock_Picks_2025-12-29.xlsx...
✅ Success! Excel file created.
   - Value Picks: 8
   - Income Picks: 39
   - Growth Picks: 6


In [8]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 11 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...


C:\Users\James\AppData\Local\Temp\ipykernel_20432\4051987151.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.49,-1.10,17.68,-46.32,1.40,21.75,0.67,3.18
0,ADMA,19.28,-1.33,17.90,-24.89,1.00,30.00,0.41,3.26
9,SEI,44.65,-0.93,32.46,-21.72,1.17,65.45,0.31,5.90
7,MIR,23.75,-0.92,19.89,-21.56,1.12,30.62,0.34,3.49
10,UBER,81.26,0.14,84.45,-20.33,1.47,112.40,0.33,2.41
5,FLEX,63.33,-0.35,48.21,-12.31,1.50,76.00,0.23,2.90
2,ARCC,20.20,1.00,20.48,-9.77,1.27,22.64,0.85,1.38
8,ONB,22.98,-0.35,21.40,-3.76,1.85,25.92,0.70,2.14
1,APG,39.57,0.43,31.34,-2.49,1.45,43.40,0.29,1.92
3,BANC,19.70,0.15,15.35,-1.84,1.27,22.14,0.44,2.18
